In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Alopecia/GSE148346'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"JAK3/TEC and TYK2/JAK1 inhibitors demonstrate significant improvement in scalp alopecia areata biomarkers"
!Series_summary	"We present the biopsy sub-study results from the first randomized, placebo-controlled clinical trial in patients with alopecia areata (AA) with ≥50% scalp hair loss and ≤7 years since the last AA episode. In this sub-study, we evaluated the molecular responses to PF-06651600, an oral inhibitor of JAK3 and the tyrosine kinase expressed in hepatocellular carcinoma (TEC) kinase family, and PF-06700841, an oral TYK2/JAK1 inhibitor, versus placebo in nonlesional and lesional scalp biopsies of biopsy samples from patients with AA."
!Series_overall_design	"This is a novel design, phase 2a, multicenter study that evaluates the efficacy, safety, and tolerability of PF-06651600 and PF-06700841 versus placebo in patients with AA. The biopsy sub-study took place during the randomized, double-blind initial 24 weeks of the trial. 46 patien

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = age_row = gender_row = None  # set to different values when applicable

def convert_trait(value):
    # In this case, trait data seems not explicitly available
    return None

def convert_age(value):
    try:
        _, val = value.split(':')
        return float(val.strip())
    except ValueError:
        return None

def convert_gender(value):
    _, val = value.split(':')
    val = val.strip().lower()
    if val == 'male':
        return 1
    elif val == 'female':
        return 0
    return None

# Save cohort information
save_cohort_info('GSE148346', './preprocessed/Alopecia/cohort_info.json', is_gene_available, trait_row is not None)

# Since trait_row is None, we skip the clinical feature extraction step.
